In [1]:
import os
import shutil
import random

# Set paths
dataset_path = r"D:\Real Vs Monster_Telephon"
output_path = r"D:\Real Vs Monster_Telephon_Split"

# Split ratios
train_ratio = 0.70
test_ratio = 0.15
val_ratio = 0.15

# Create output directories
for split in ["train", "test", "val"]:
    os.makedirs(os.path.join(output_path, split), exist_ok=True)

# Loop through each class folder
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)

    if not os.path.isdir(class_path):
        continue

    # Create class folders inside train/test/val
    for split in ["train", "test", "val"]:
        os.makedirs(os.path.join(output_path, split, class_name), exist_ok=True)

    # List and shuffle files
    files = os.listdir(class_path)
    random.shuffle(files)

    total = len(files)
    train_end = int(total * train_ratio)
    test_end = train_end + int(total * test_ratio)

    train_files = files[:train_end]
    test_files = files[train_end:test_end]
    val_files = files[test_end:]

    # Copy files to split folders
    for f in train_files:
        shutil.copy(os.path.join(class_path, f),
                    os.path.join(output_path, "train", class_name))

    for f in test_files:
        shutil.copy(os.path.join(class_path, f),
                    os.path.join(output_path, "test", class_name))

    for f in val_files:
        shutil.copy(os.path.join(class_path, f),
                    os.path.join(output_path, "val", class_name))

    print(f"[DONE] {class_name} → train:{len(train_files)}  test:{len(test_files)}  val:{len(val_files)}")

print("\nDataset successfully split into Train/Test/Val!")


[DONE] Barishal Real → train:630  test:135  val:136
[DONE] Barishal_monster_telephone → train:630  test:135  val:136
[DONE] Chapai Real → train:700  test:150  val:150
[DONE] Chapai_monster_telephone → train:700  test:150  val:150
[DONE] Chittagong Real → train:1106  test:237  val:237
[DONE] Chittagong_monster_telephone → train:1106  test:237  val:237
[DONE] Habiganj Real → train:739  test:158  val:160
[DONE] Habiganj_monster_telephone → train:739  test:158  val:160
[DONE] kishoreganj Real → train:1289  test:276  val:277
[DONE] kishoreganj_monster_telephone → train:1289  test:276  val:277
[DONE] Kustia Real → train:700  test:150  val:150
[DONE] Kustia_monster_telephone → train:700  test:150  val:150
[DONE] Naoga Real → train:700  test:150  val:150
[DONE] Naoga_monster_telephone → train:700  test:150  val:150
[DONE] Narail Real → train:1169  test:250  val:252
[DONE] Narail_monster_telephone → train:1169  test:250  val:252
[DONE] Narsingdi Real → train:863  test:185  val:186
[DONE] Narsin

In [1]:
# ===================== AASIST-Lite (Spectro-Temporal) FULL TRAIN+EVAL CODE =====================
# Works with your folder structure:
# D:\RealVsMonster_Split\train\<class>\audio.*
# D:\RealVsMonster_Split\val\<class>\audio.*
# D:\RealVsMonster_Split\test\<class>\audio.*
#
# Output:
# - best_aasist_lite.pth
# - confusion_matrix.png
# - roc_multiclass.png
# - pr_multiclass.png
# - loss_curve.png
# - acc_curve.png
# - waveform.png / spectrogram.png / chromagram.png
# - pca_embeddings.png / tsne_embeddings.png
# - classification report printed in console

import os, random
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve
from sklearn.preprocessing import label_binarize
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tqdm import tqdm

# ---------------- CONFIG ----------------
DATASET_ROOT = r"D:\Real Vs Monster_Telephon_Split"
SAMPLE_RATE  = 16000

# Feature: 2-channel -> [LogMel, Linear-Fbank(log)]  (spectral)
N_MELS       = 64
N_LINFB      = 64
N_FFT        = 1024
HOP_LENGTH   = 256
MAX_FRAMES   = 256

# Training
BATCH_SIZE   = 16
EPOCHS       = 30
LR           = 2e-4
RANDOM_SEED  = 42

# SpecAugment
USE_SPECAUG      = True
TIME_MASK_PARAM  = 24
FREQ_MASK_PARAM  = 6

# Audio safety
MIN_AUDIO_SAMPLES = 2048
MIN_RMS = 1e-4

torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------- CLASS NAMES ----------------
train_base = os.path.join(DATASET_ROOT, "train")
CLASS_NAMES = sorted([d for d in os.listdir(train_base) if os.path.isdir(os.path.join(train_base, d))])
NUM_CLASSES = len(CLASS_NAMES)
print("Classes:", NUM_CLASSES)
print(CLASS_NAMES)

# ---------------- HELPERS ----------------
def pad_trunc_2d(X, max_frames):
    # X: (F, T)
    if X.shape[1] < max_frames:
        pad = np.zeros((X.shape[0], max_frames - X.shape[1]), dtype=np.float32)
        X = np.concatenate([X, pad], axis=1)
    else:
        X = X[:, :max_frames]
    return X

def zscore_norm(X, eps=1e-6):
    mu = float(X.mean())
    std = float(X.std())
    return (X - mu) / (std + eps)

def safe_load_and_trim(path):
    y, sr = librosa.load(path, sr=SAMPLE_RATE, mono=True)
    y, _ = librosa.effects.trim(y, top_db=30)

    if y is None or len(y) < MIN_AUDIO_SAMPLES:
        return None, sr

    rms = float(np.sqrt(np.mean(y**2) + 1e-12))
    if rms < MIN_RMS:
        return None, sr

    return y, sr

# ---------------- FEATURE EXTRACTION ----------------
def extract_logmel(y, sr):
    mel = librosa.feature.melspectrogram(
        y=y, sr=sr, n_mels=N_MELS, n_fft=N_FFT, hop_length=HOP_LENGTH, power=2.0
    )
    mel = librosa.power_to_db(mel, ref=np.max)
    mel = zscore_norm(mel).astype(np.float32)
    mel = pad_trunc_2d(mel, MAX_FRAMES)
    return mel  # (64, T)

def extract_log_linear_fbank(y, sr):
    """
    Manual linear-frequency filterbank energies (log)  (no librosa.filters.linear dependency)
    """
    S = np.abs(librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH))**2  # (F, T)
    F_bins = S.shape[0]

    freqs = np.linspace(0, sr/2, F_bins, dtype=np.float32)
    edges = np.linspace(0, sr/2, N_LINFB + 2, dtype=np.float32)

    fb = np.zeros((N_LINFB, F_bins), dtype=np.float32)
    for m in range(N_LINFB):
        f_left, f_center, f_right = edges[m], edges[m+1], edges[m+2]
        left  = (freqs - f_left) / (f_center - f_left + 1e-9)
        right = (f_right - freqs) / (f_right - f_center + 1e-9)
        fb[m] = np.maximum(0.0, np.minimum(left, right))

    E = np.dot(fb, S) + 1e-8
    E = np.log(E)

    E = zscore_norm(E).astype(np.float32)
    E = pad_trunc_2d(E, MAX_FRAMES)
    return E  # (64, T)

def extract_features(path):
    y, sr = safe_load_and_trim(path)
    if y is None:
        mel = np.zeros((N_MELS, MAX_FRAMES), dtype=np.float32)
        lfb = np.zeros((N_LINFB, MAX_FRAMES), dtype=np.float32)
    else:
        mel = extract_logmel(y, sr)
        lfb = extract_log_linear_fbank(y, sr)
    X = np.stack([mel, lfb], axis=0).astype(np.float32)  # (2, 64, T)
    return X

# ---------------- SPECAUGMENT ----------------
def spec_augment(x, time_mask_param=24, freq_mask_param=6):
    # x: torch tensor (C,F,T) ; masks apply all channels
    if not USE_SPECAUG:
        return x
    C, F, T = x.shape

    f = random.randint(0, min(freq_mask_param, F))
    f0 = random.randint(0, max(0, F - f))
    if f > 0:
        x[:, f0:f0+f, :] = 0

    t = random.randint(0, min(time_mask_param, T))
    t0 = random.randint(0, max(0, T - t))
    if t > 0:
        x[:, :, t0:t0+t] = 0

    return x

# ---------------- DATASET ----------------
class SpecDataset(Dataset):
    def __init__(self, root, split):
        base = os.path.join(root, split)
        self.split = split
        self.paths, self.labels = [], []
        self.cls_to_idx = {c: i for i, c in enumerate(CLASS_NAMES)}

        for cls in CLASS_NAMES:
            folder = os.path.join(base, cls)
            if not os.path.isdir(folder):
                continue
            for f in os.listdir(folder):
                if f.lower().endswith((".mp3", ".wav", ".ogg", ".flac", ".m4a")):
                    self.paths.append(os.path.join(folder, f))
                    self.labels.append(self.cls_to_idx[cls])

        print(f"{split} set: {len(self.paths)} files")
        self.cache = {}  # cache val/test

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        label = self.labels[idx]

        if self.split != "train" and path in self.cache:
            feat = self.cache[path]
        else:
            feat = extract_features(path)
            if self.split != "train":
                self.cache[path] = feat

        x = torch.tensor(feat)  # (2,64,T)
        if self.split == "train":
            x = spec_augment(x, TIME_MASK_PARAM, FREQ_MASK_PARAM)

        return x, torch.tensor(label, dtype=torch.long), path

train_ds = SpecDataset(DATASET_ROOT, "train")
val_ds   = SpecDataset(DATASET_ROOT, "val")
test_ds  = SpecDataset(DATASET_ROOT, "test")

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=1,          shuffle=False, num_workers=0, pin_memory=True)

# ---------------- MODEL (AASIST-Lite): CNN(spectral) + Transformer(temporal) + AttnPool ----------------
class AttentivePool(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.attn = nn.Sequential(
            nn.Linear(d, d//2),
            nn.Tanh(),
            nn.Linear(d//2, 1)
        )
    def forward(self, x):   # (B,T,D)
        w = torch.softmax(self.attn(x), dim=1)  # (B,T,1)
        return (w * x).sum(dim=1)               # (B,D)

class AASISTLite(nn.Module):
    """
    Input: (B, 2, 64, T)
    CNN -> spectral patterns
    Transformer -> temporal patterns
    """
    def __init__(self, num_classes, d_model=256, nhead=4, num_layers=2, dropout=0.2):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(2, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.MaxPool2d((2,2)),  # F/2, T/2

            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d((2,2)),  # F/4, T/4

            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d((2,2)),  # F/8, T/8
        )

        f_after = N_MELS // 8  # 64 -> 8
        self.proj = nn.Linear(128 * f_after, d_model)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=d_model*4,
            dropout=dropout, batch_first=True, activation="gelu", norm_first=True
        )
        self.temporal = nn.TransformerEncoder(enc_layer, num_layers=num_layers)

        self.pool = AttentivePool(d_model)
        self.head = nn.Sequential(
            nn.Linear(d_model, 256), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        z = self.cnn(x)                      # (B,128,F',T')
        B, C, F, T = z.shape
        z = z.permute(0, 3, 1, 2).contiguous()  # (B,T,C,F)
        z = z.view(B, T, C*F)                   # (B,T,128*F')
        z = self.proj(z)                        # (B,T,d_model)

        z = self.temporal(z)                    # (B,T,d_model)  (temporal features)
        emb = self.pool(z)                      # (B,d_model)
        logits = self.head(emb)                 # (B,num_classes)
        return logits, emb

model = AASISTLite(num_classes=NUM_CLASSES).to(device)
print(model)

# ---------------- LOSS/OPT ----------------
counts = np.bincount(train_ds.labels, minlength=NUM_CLASSES).astype(np.float32)
w = (counts.sum() / (counts + 1e-6))
w = w / w.mean()
class_weights = torch.tensor(w, dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=2)

use_amp = (device.type == "cuda")
scaler = torch.amp.GradScaler('cuda', enabled=use_amp)

def acc_from_logits(logits, y):
    return (torch.argmax(logits, 1) == y).float().mean().item()

# ---------------- TRAIN ----------------
train_losses, val_losses = [], []
train_accs, val_accs = [], []
best_val = -1.0

for epoch in range(EPOCHS):
    model.train()
    tr_loss_sum, tr_acc_sum, tr_n = 0.0, 0.0, 0

    for x, y, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [train]"):
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with torch.amp.autocast(device_type='cuda', enabled=use_amp):
            logits, _ = model(x)
            loss = criterion(logits, y)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        scaler.step(optimizer)
        scaler.update()

        bs = y.size(0)
        tr_loss_sum += loss.item() * bs
        tr_acc_sum  += acc_from_logits(logits.detach(), y) * bs
        tr_n += bs

    train_loss = tr_loss_sum / tr_n
    train_acc  = tr_acc_sum / tr_n
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    model.eval()
    va_loss_sum, va_acc_sum, va_n = 0.0, 0.0, 0
    with torch.no_grad():
        for x, y, _ in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [val]"):
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)
            logits, _ = model(x)
            loss = criterion(logits, y)

            bs = y.size(0)
            va_loss_sum += loss.item() * bs
            va_acc_sum  += acc_from_logits(logits, y) * bs
            va_n += bs

    val_loss = va_loss_sum / va_n
    val_acc  = va_acc_sum / va_n
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    scheduler.step(val_acc)

    print(f"Epoch {epoch+1:02d} | Train Loss {train_loss:.4f} Acc {train_acc:.4f} | Val Loss {val_loss:.4f} Acc {val_acc:.4f}")

    if val_acc > best_val + 1e-6:
        best_val = val_acc
        torch.save(model.state_dict(), "best_aasist_lite.pth")

print("Training done. Best Val Acc:", best_val)
print("Saved best weights to best_aasist_lite.pth")

# ---------------- TEST + REPORTS + CURVES ----------------
model.load_state_dict(torch.load("best_aasist_lite.pth", map_location=device))
model.eval()

y_true, y_pred = [], []
probs_all = []
embs, emb_labels = [], []

with torch.no_grad():
    for x, y, _ in tqdm(test_loader, desc="Testing"):
        x = x.to(device)
        logits, emb = model(x)
        prob = torch.softmax(logits, dim=1).cpu().numpy()[0]
        pred = int(np.argmax(prob))

        y_true.append(int(y.item()))
        y_pred.append(pred)
        probs_all.append(prob)
        embs.append(emb.cpu().numpy()[0])
        emb_labels.append(int(y.item()))

y_true = np.array(y_true)
y_pred = np.array(y_pred)
probs_all = np.array(probs_all)
embs = np.array(embs)
emb_labels = np.array(emb_labels)

print("\n================ CLASSIFICATION REPORT ================\n")
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12, 10))
plt.imshow(cm, interpolation="nearest")
plt.title("Confusion Matrix (AASIST-Lite)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.colorbar()
plt.tight_layout()
plt.savefig("confusion_matrix.png", dpi=200)
plt.close()

# ROC + PR (multi-class OVR)
Y_bin = label_binarize(y_true, classes=list(range(NUM_CLASSES)))

# ROC
plt.figure(figsize=(10, 7))
for i in range(NUM_CLASSES):
    fpr, tpr, _ = roc_curve(Y_bin[:, i], probs_all[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{CLASS_NAMES[i]} (AUC={roc_auc:.2f})")
plt.plot([0, 1], [0, 1], "k--")
plt.title("Multi-class ROC Curve (AASIST-Lite)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(fontsize=7, loc="lower right")
plt.tight_layout()
plt.savefig("roc_multiclass.png", dpi=200)
plt.close()

# PR
plt.figure(figsize=(10, 7))
for i in range(NUM_CLASSES):
    prec, rec, _ = precision_recall_curve(Y_bin[:, i], probs_all[:, i])
    pr_auc = auc(rec, prec)
    plt.plot(rec, prec, label=f"{CLASS_NAMES[i]} (AUC={pr_auc:.2f})")
plt.title("Multi-class Precision-Recall Curve (AASIST-Lite)")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(fontsize=7, loc="lower left")
plt.tight_layout()
plt.savefig("pr_multiclass.png", dpi=200)
plt.close()

# Train/Val curves
plt.figure()
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.title("Training vs Validation Loss (AASIST-Lite)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.tight_layout()
plt.savefig("loss_curve.png", dpi=200)
plt.close()

plt.figure()
plt.plot(train_accs, label="Train Acc")
plt.plot(val_accs, label="Val Acc")
plt.title("Training vs Validation Accuracy (AASIST-Lite)")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.tight_layout()
plt.savefig("acc_curve.png", dpi=200)
plt.close()

# Waveform + Spectrogram + Chromagram (one test file)
sample_path = test_ds.paths[0] if len(test_ds.paths) else None
if sample_path:
    y, sr = librosa.load(sample_path, sr=SAMPLE_RATE, mono=True)

    plt.figure(figsize=(12, 3))
    plt.plot(np.linspace(0, len(y)/sr, len(y)), y)
    plt.title("Waveform")
    plt.xlabel("Time (s)")
    plt.tight_layout()
    plt.savefig("waveform.png", dpi=200)
    plt.close()

    D = librosa.amplitude_to_db(np.abs(librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH)), ref=np.max)
    plt.figure(figsize=(12, 4))
    librosa.display.specshow(D, sr=sr, hop_length=HOP_LENGTH, x_axis="time", y_axis="hz")
    plt.colorbar(format="%+0.0f dB")
    plt.title("Spectrogram (dB)")
    plt.tight_layout()
    plt.savefig("spectrogram.png", dpi=200)
    plt.close()

    chroma = librosa.feature.chroma_stft(y=y, sr=sr, n_fft=N_FFT, hop_length=HOP_LENGTH, tuning=0.0)
    plt.figure(figsize=(12, 3))
    librosa.display.specshow(chroma, sr=sr, hop_length=HOP_LENGTH, x_axis="time", y_axis="chroma")
    plt.colorbar()
    plt.title("Chromagram")
    plt.tight_layout()
    plt.savefig("chromagram.png", dpi=200)
    plt.close()

# PCA / t-SNE embeddings
pca = PCA(n_components=2, random_state=RANDOM_SEED)
Zp = pca.fit_transform(embs)
plt.figure(figsize=(8, 6))
sc = plt.scatter(Zp[:, 0], Zp[:, 1], c=emb_labels, s=10)
plt.title("PCA of AASIST-Lite Embeddings")
plt.colorbar(sc)
plt.tight_layout()
plt.savefig("pca_embeddings.png", dpi=200)
plt.close()

tsne = TSNE(n_components=2, random_state=RANDOM_SEED, init="pca", learning_rate="auto")
Zt = tsne.fit_transform(embs)
plt.figure(figsize=(8, 6))
sc = plt.scatter(Zt[:, 0], Zt[:, 1], c=emb_labels, s=10)
plt.title("t-SNE of AASIST-Lite Embeddings")
plt.colorbar(sc)
plt.tight_layout()
plt.savefig("tsne_embeddings.png", dpi=200)
plt.close()

print("\nSaved figures:")
print("confusion_matrix.png, roc_multiclass.png, pr_multiclass.png, loss_curve.png, acc_curve.png")
print("waveform.png, spectrogram.png, chromagram.png, pca_embeddings.png, tsne_embeddings.png")
print("Best model: best_aasist_lite.pth")
# ==============================================================================================


Device: cuda
Classes: 30
['Barishal Real', 'Barishal_monster_telephone', 'Chapai Real', 'Chapai_monster_telephone', 'Chittagong Real', 'Chittagong_monster_telephone', 'Habiganj Real', 'Habiganj_monster_telephone', 'Kustia Real', 'Kustia_monster_telephone', 'Naoga Real', 'Naoga_monster_telephone', 'Narail Real', 'Narail_monster_telephone', 'Narsingdi Real', 'Narsingdi_monster_telephone', 'Rajshahi Real', 'Rajshahi_monster_telephone', 'Rangpur Real', 'Rangpur_monster_telephone', 'Sandwip Real', 'Sandwip_monster_telephone', 'Sylhet Real', 'Sylhet_monster_telephone', 'Tangail Real', 'Tangail_monster_telephone', 'kishoreganj Real', 'kishoreganj_monster_telephone', 'pabna Real', 'pabna_monster_telephone']
train set: 27944 files
val set: 6016 files
test set: 5980 files
AASISTLite(
  (cnn): Sequential(
    (0): Conv2d(2, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPoo

C:\Users\acer\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(
Epoch 1/30 [val]: 100%|██████████████████████████████████████████████████████████████| 376/376 [04:33<00:00,  1.37it/s]


Epoch 01 | Train Loss 1.5016 Acc 0.4509 | Val Loss 1.1170 Acc 0.6059


Epoch 2/30 [val]: 100%|█████████████████████████████████████████████████████████████| 376/376 [00:02<00:00, 151.69it/s]


Epoch 02 | Train Loss 0.9620 Acc 0.6243 | Val Loss 0.9131 Acc 0.6740


...

Epoch 29 | Train Loss 0.0646 Acc 0.9735 | Val Loss 0.3179 Acc 0.9317


Epoch 30/30 [val]: 100%|████████████████████████████████████████████████████████████| 376/376 [00:02<00:00, 143.94it/s]


Epoch 30 | Train Loss 0.0621 Acc 0.9752 | Val Loss 0.3298 Acc 0.9307
Training done. Best Val Acc: 0.9316821808510638
Saved best weights to best_aasist_lite.pth


C:\Users\acer\AppData\Local\Temp\ipykernel_23796\2623449860.py:358: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_aasist_lite.pth", ma


================ CLASSIFICATION REPORT ================

                               precision    recall  f1-score   support

                Barishal Real     0.9470    0.9259    0.9363       135
   Barishal_monster_telephone     0.8235    0.8296    0.8266       135
                  Chapai Real     0.9934    1.0000    0.9967       150
     Chapai_monster_telephone     0.9933    0.9933    0.9933       150
              Chittagong Real     0.9309    0.9662    0.9482       237
 Chittagong_monster_telephone     0.8051    0.9241    0.8605       237
                Habiganj Real     0.8111    0.9241    0.8639       158
   Habiganj_monster_telephone     0.8261    0.7215    0.7703       158
                  Kustia Real     1.0000    1.0000    1.0000       150
     Kustia_monster_telephone     0.9933    0.9933    0.9933       150
                   Naoga Real     1.0000    1.0000    1.0000       150
      Naoga_monster_telephone     0.9933    0.9867    0.9900       150
                  